# Test

In [8]:
import os
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras import utils


# 데이터 셋 다운

In [14]:
# 데이터 셋 다운 받기
data_set = tf.keras.utils.get_file(fname="imdb.tar.gz",
                                    origin = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz',
                                    extract = True)

84125825/84125825 [==============================] - 78s 1us/step


# Train, Test 데이터 프레임 만들기

##### 함수 정의하기

In [20]:
# 데이터 읽고 데이터 프레임 만들기 함수
def directory_data(directory):
    data={}
    data['review']=[]
    
    for file_path in os.listdir(directory):
        with open(os.path.join(directory, file_path), 'r', encoding='utf=8') as file: # 파일 읽기
            data['review'].append(file.read()) # data['review']에 내용 저장하기
    return pd.DataFrame.from_dict(data) # 데이터 프레임 반환하기

# 긍정, 부정 데이터 프레임 만들기
def data(directory):
    pos_df = directory_data(os.path.join(directory, 'pos')) # 긍정적 데이터 프레임
    neg_df = directory_data(os.path.join(directory, 'neg')) # 부정적 데이터 프레임
    
    pos_df['sentiment']=1
    neg_df['sentiment']=0
    
    return pd.concat([pos_df, neg_df])

##### Train, Test 데이터 프레임 저장하기

In [ ]:
train_df = data(os.path.join(os.path.dirname(data_set),"aclImdb","train"))
test_df = data(os.path.join(os.path.dirname(data_set),"aclImdb","test"))

##### 분리된 데이터 확인하기